## RESTful

### Restful 1.1

In [2]:
from flask import Flask, jsonify, request
import mysql.connector
import pandas as pd
from mysql.connector import Error
from flask_httpauth import HTTPBasicAuth

auth = HTTPBasicAuth()
app = Flask(__name__)

global invitado
invitado = None

publicConnection = mysql.connector.connect(host='localhost',
                                     database='ssdd',
                                     user='root',
                                     password='h0st1P!')


@auth.verify_password
def verify_password(username, password):
    
    global invitado
    
    with open("userpass.txt", 'r') as file:
        data = file.readlines()
        users = dict([i.split(':') for i in data])
        
    if username in users and password == users[username][:-1]: #If user and password in list
        
        if username == "invitado":
            invitado = True
        else:
            invitado = False
        
        return True
    return False

In [2]:
# 1. Consultar la cantidad de fechas disponibles en la base de datos
## http://127.0.0.1:6878/api/v1.0/fechas

@app.route('/api/v1.0/fechas', methods=['GET'])
def get_fechas():
    
    if request.method == 'GET':
        df = pd.read_sql_query('SELECT COUNT(DISTINCT fecha) AS cantidad_fechas_disponibles FROM bicimad', publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 2. Consultar los códigos de las “base_stations” de origen 

# http://127.0.0.1:6878/api/v1.0/BSOrigen
    
@app.route('/api/v1.0/BSOrigen', methods=['GET'])
def get_bsorigen():
    
    if request.method == 'GET':
        df = pd.read_sql_query('SELECT idunplug_station FROM bicimad', publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 3. Consultar los códigos de las “base_stations” de destino

# http://127.0.0.1:6878/api/v1.0/BSDestino
    
@app.route('/api/v1.0/BSDestino', methods=['GET'])
def get_bsdestino():
    
    if request.method == 'GET':
        df = pd.read_sql_query('SELECT idplug_station FROM bicimad', publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 4. Recuperar todos los movimientos para un determinado día

# http://127.0.0.1:6878/api/v1.0/movimientos?fecha=2019-05-31
    
@app.route('/api/v1.0/movimientos', methods=['GET'])
def get_movfecha():
    
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}'"
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 5. Recuperar todos los movimientos para un determinado día que hagan un
# trayecto iniciado en una determinada base de origen 

# http://127.0.0.1:6878/api/v1.0/movimientosBSorig?fecha=2019-05-31&bsorigen=53
    
@app.route('/api/v1.0/movimientosBSorig', methods=['GET'])
def get_movFechaBsorigen():
    
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsorigen = request.args.get('bsorigen')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}'"
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 6. Recuperar todos los movimientos para un determinado día que hagan un trayecto
# finalizado en una determinada base de destino

# http://127.0.0.1:6878/api/v1.0/movimientosBSdest?fecha=2019-05-31&bsdestino=53
    
@app.route('/api/v1.0/movimientosBSDest', methods=['GET'])
def get_movFechaBdestino():
    
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsdestino = request.args.get('bsdestino')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idplug_station = '{bsdestino}'"
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 7. Recuperar todos los movimientos para un determinado día que hagan un trayecto
# iniciado en una determinada base de origen y finalizado en una determinada base de
# destino

# http://127.0.0.1:6878/api/v1.0/movimientosBSDestBsorig?fecha=2019-05-31&bsorigen=129&bsdestino=64
    
@app.route('/api/v1.0/movimientosBSDestBsorig', methods=['GET'])
def get_movFechaBdestinoBorig():
    
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsorigen = request.args.get('bsorigen')
        bsdestino = request.args.get('bsdestino')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' AND idplug_station = '{bsdestino}'"
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 8. Recuperar todos los movimientos para un determinado día que hagan un trayecto
# iniciado en una determinada base de origen y finalizado en una determinada base de
# destino y con una duración de trayecto inferior o superior a un determinado valor 

# http://127.0.0.1:6878/api/v1.0/movimientosBdestBorigTtime?fecha=2019-05-31&bsorigen=77&bsdestino=171&mindur=10&maxdur=600
    
@app.route('/api/v1.0/movimientosBdestBorigTtime', methods=['GET'])
def get_movFechaBdestBorigTTime():
    
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsorigen = request.args.get('bsorigen')
        bsdestino = request.args.get('bsdestino')
        max_dur = request.args.get('maxdur')
        min_dur = request.args.get('mindur')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        if int(max_dur) - int(min_dur) < 0:
            return "La duracion max debe ser mayor que la duracion min", 400
        
        query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' AND idplug_station = '{bsdestino}' AND travel_time >= '{min_dur}' AND travel_time <= '{max_dur}'"
        print(query)
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
        
    return out

# 9. Introducir una nueva línea en la base de datos con la información de fecha, ageRange,
# user_type, idunplug_station, idplug_station, idunplug_base, idplug_base y travel_time.
# El campo Fichero debe contener el valor “000000” para todas las entradas
# introducidas por este mecanismo valor 

# http://127.0.0.1:6878/api/v1.0/newline
    
@app.route('/api/v1.0/newline', methods=['POST', 'GET'])
@auth.login_required
def get_newdata_posted():
    if request.method == 'GET':
        return '''<form method="POST">
                  Fecha: <input type="Date" name="Fecha"><br>
                  ageRange: <input type="number" name="ageRange"><br>
                  user_type: <input type="number" name="user_type"><br>
                  idunplug_station: <input type="number" name="idunplug_station"><br>
                  idplug_station: <input type="number" name="idplug_station"><br>
                  idunplug_base: <input type="number" name="idunplug_base"><br>
                  idplug_base: <input type="number" name="idplug_base"><br>
                  travel_time: <input type="number" name="travel_time"><br>
                  <input type="submit" value="Submit"><br>
              </form>'''
    elif  request.method == 'POST':
        fecha = request.form.get('Fecha')
        agerange = request.form.get('ageRange')
        user_type = request.form.get('user_type')
        bsunplug_st = request.form.get('idunplug_station')
        bsplug_st = request.form.get('idplug_station')
        bsunplug_bs = request.form.get('idunplug_base')
        bsplug_bs  = request.form.get('idplug_base')
        travel_time = request.form.get('travel_time')
        
        query = f"INSERT INTO bicimad (fecha, ageRange, user_type, idunplug_station, idplug_station, idunplug_base, idplug_base, travel_time, Fichero)VALUES ('{fecha}', '{agerange}', '{user_type}', '{bsunplug_st}', '{bsplug_st}', '{bsunplug_bs}', '{bsplug_st}', '{travel_time}', '000000')"
        with publicConnection.cursor() as cursor:
            cursor.execute(query)
            
        publicConnection.commit() 
        return "Form submited correctly"
        
    return 0

In [ ]:
if __name__ == '__main__':
    app.run(port=6878)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6878
Press CTRL+C to quit
C:\Users\Daniel\AppData\Local\Temp\ipykernel_2616\1890726817.py:116: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, publicConnection)
127.0.0.1 - - [02/Mar/2024 14:42:30] "GET /api/v1.0/movimientosBSDestBsorig?fecha=2019-05-31&bsorigen=77&bsdestino=171 HTTP/1.1" 200 -
C:\Users\Daniel\AppData\Local\Temp\ipykernel_2616\1890726817.py:116: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, publicConnection)
127.0.0.1 - - [02/Mar/2024 14:46:16] "GET /api/v1.0/movimientosBSDestBsorig?fecha=2019-05-31&bsorigen=129&bsdestino=64 HTTP/1.1" 200 -


### Restful 1.2

In [2]:
# 1.Implementar una función de tipo PUT que permita actualizar el campo del tiempo de
# trayecto (travel_time) para una determinada entrada con valores fecha, ageRange,
# user_type, idunplug_station, idplug_station, idunplug_base, idplug_base.

# http://127.0.0.1:6878/api/v1.0/newTime
    
@app.route('/api/v1.0/newTime', methods=['POST', 'GET'])
def updateTime():
    
    # Los formularios solo aceptan los métodos GET y POST
    if request.method == 'GET':
        return '''<form method="POST">
                  Fecha: <input type="Date" name="Fecha"><br>
                  ageRange: <input type="number" name="ageRange"><br>
                  user_type: <input type="number" name="user_type"><br>
                  idunplug_station: <input type="number" name="idunplug_station"><br>
                  idplug_station: <input type="number" name="idplug_station"><br>
                  idunplug_base: <input type="number" name="idunplug_base"><br>
                  idplug_base: <input type="number" name="idplug_base"><br>
                  Nuevo tiempo de trayecto: <input type="number" name="travel_time"><br>
                  <input type="submit" value="Submit"><br>
              </form>'''
    elif  request.method == 'POST':
        fecha = request.form.get('Fecha')
        agerange = request.form.get('ageRange')
        user_type = request.form.get('user_type')
        bsunplug_st = request.form.get('idunplug_station')
        bsplug_st = request.form.get('idplug_station')
        bsunplug_bs = request.form.get('idunplug_base')
        bsplug_bs  = request.form.get('idplug_base')
        travel_time = request.form.get('travel_time')
        
        query = f"UPDATE bicimad SET travel_time = '{travel_time}' WHERE fecha = '{fecha}' AND ageRange = '{agerange}' AND user_type = '{user_type}' AND idunplug_station = '{bsunplug_st}' AND idplug_station = '{bsplug_st}' AND idunplug_base = '{bsunplug_bs}' AND idplug_base = '{bsplug_bs}'"
        with publicConnection.cursor() as cursor:
            cursor.execute(query)
            
        publicConnection.commit()
        return "Campo actualizado correctamente"
        
    return 0

# 2.Implementar una función de tipo DELETE que permita eliminar campos de la base de
# datos pasando una combinación de cualquiera de los parámetros fecha, ageRange,
# user_type, idunplug_station, idplug_station, idunplug_base, idplug_base, fichero. Es
# decir, si se pasa sólo la fecha, se eliminarán todas las entradas con esa fecha, si se
# pasan el idunplug_station y el idplug_station se eliminarán todos los trayectos entre
# esas estaciones de origen y destino y si se pasan valores para todas las sólo se
# eliminará la única entrada que coincida con esa combinación.

# http://127.0.0.1:6878/api/v1.0/delete?fecha=2019-06-25&travel_time=971&ageRange=4
    
@app.route('/api/v1.0/delete', methods=['GET'])
def deleteRow():
    
    print(request.method)
    
    if request.method == 'GET':
        
        fecha = request.args.get('fecha')
        age_range = request.args.get('ageRange')
        user_type = request.args.get('user_type')
        idunplug_station = request.args.get('idunplug_station')
        idplug_station = request.args.get('idplug_station')
        idunplug_base = request.args.get('idunplug_base')
        idplug_base = request.args.get('idplug_base')
        travel_time = request.args.get('travel_time')
        
        whereQuery = []
        if fecha:
            whereQuery.append(f"fecha = '{fecha}'")
        if age_range:
            whereQuery.append(f"ageRange = '{age_range}'")
        if user_type:
            whereQuery.append(f"user_type = '{user_type}'")
        if idunplug_station:
            whereQuery.append(f"idunplug_station = '{idunplug_station}'")
        if idplug_station:
            whereQuery.append(f"idplug_station = '{idplug_station}'")
        if idunplug_base:
            whereQuery.append(f"idunplug_base = '{idunplug_base}'")
        if idplug_base:
            whereQuery.append(f"idplug_base = '{idplug_base}'")
            
        query = f"DELETE FROM bicimad WHERE "
        if whereQuery:
            query += " AND ".join(whereQuery)

        with publicConnection.cursor() as cursor:
            cursor.execute(query)

        publicConnection.commit()
        return "Campo eliminado correctamente"
        
    return 0

In [3]:
# 3. Implementar un sistema por el cual usar el API sin autenticación para realizar llamadas
# de tipo GET sólo devuelva 10 elementos de cualquier consulta mientras que si se hace
# la llamada GET con el usuario de administración devuelve todos los elementos que
# coincidan con esa consulta.

# 1. Consultar la cantidad de fechas disponibles en la base de datos
## http://127.0.0.1:6878/api/v1.0/fechas

@app.route('/api/v1.0/fechas', methods=['GET'])
@auth.login_required
def get_fechas():
    
    global invitado
    if request.method == 'GET':
        
        print(invitado)
        
        if invitado == True:
            
            query = 'SELECT COUNT(DISTINCT fecha) AS cantidad_fechas_disponibles FROM bicimad LIMIT 10'
            
        else:
            
            query = 'SELECT COUNT(DISTINCT fecha) AS cantidad_fechas_disponibles FROM bicimad'
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 2. Consultar los códigos de las “base_stations” de origen 

# http://127.0.0.1:6878/api/v1.0/BSOrigen
    
@app.route('/api/v1.0/BSOrigen', methods=['GET'])
@auth.login_required
def get_bsorigen():
    
    global invitado
    if request.method == 'GET':
        
        if invitado == True:
            
            query = 'SELECT idunplug_station FROM bicimad LIMIT 10'
            
        else:
            
            query = 'SELECT idunplug_station FROM bicimad'
        
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 3. Consultar los códigos de las “base_stations” de destino

# http://127.0.0.1:6878/api/v1.0/BSDestino
    
@app.route('/api/v1.0/BSDestino', methods=['GET'])
@auth.login_required
def get_bsdestino():
    
    global invitado
    if request.method == 'GET':
        
        if invitado == True:
            query = 'SELECT idplug_station FROM bicimad LIMIT 10'
        else:
            query = 'SELECT idplug_station FROM bicimad'
        
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 4. Recuperar todos los movimientos para un determinado día

# http://127.0.0.1:6878/api/v1.0/movimientos?fecha=2019-05-31
    
@app.route('/api/v1.0/movimientos', methods=['GET'])
@auth.login_required
def get_movfecha():
    
    global invitado
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        if invitado == True:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' LIMIT 10"
        else:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}'"
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 5. Recuperar todos los movimientos para un determinado día que hagan un
# trayecto iniciado en una determinada base de origen 

# http://127.0.0.1:6878/api/v1.0/movimientosBSorig?fecha=2019-05-31&bsorigen=53
    
@app.route('/api/v1.0/movimientosBSorig', methods=['GET'])
@auth.login_required
def get_movFechaBsorigen():
    
    global invitado
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsorigen = request.args.get('bsorigen')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        if invitado == True:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' LIMIT 10"
        else:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}'"
        
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 6. Recuperar todos los movimientos para un determinado día que hagan un trayecto
# finalizado en una determinada base de destino

# http://127.0.0.1:6878/api/v1.0/movimientosBSdest?fecha=2019-05-31&bsdestino=53
    
@app.route('/api/v1.0/movimientosBSDest', methods=['GET'])
@auth.login_required
def get_movFechaBdestino():
    
    global invitado
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsdestino = request.args.get('bsdestino')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        if invitado == True:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idplug_station = '{bsdestino}' LIMIT 10"
        else:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idplug_station = '{bsdestino}'"
        
        df = pd.read_sql_query(query, publicConnection)
        out=str(df.head())
    return out

# 7. Recuperar todos los movimientos para un determinado día que hagan un trayecto
# iniciado en una determinada base de origen y finalizado en una determinada base de
# destino

# http://127.0.0.1:6878/api/v1.0/movimientosBSDestBsorig?fecha=2019-05-31&bsorigen=129&bsdestino=64
    
@app.route('/api/v1.0/movimientosBSDestBsorig', methods=['GET'])
@auth.login_required
def get_movFechaBdestinoBorig():
    
    global invitado
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsorigen = request.args.get('bsorigen')
        bsdestino = request.args.get('bsdestino')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        if invitado == True:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' AND idplug_station = '{bsdestino}' LIMIT 10"
        else:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' AND idplug_station = '{bsdestino}'"
        
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
    return out

# 8. Recuperar todos los movimientos para un determinado día que hagan un trayecto
# iniciado en una determinada base de origen y finalizado en una determinada base de
# destino y con una duración de trayecto inferior o superior a un determinado valor 

# http://127.0.0.1:6878/api/v1.0/movimientosBdestBorigTtime?fecha=2019-05-31&bsorigen=77&bsdestino=171&mindur=10&maxdur=600
    
@app.route('/api/v1.0/movimientosBdestBorigTtime', methods=['GET'])
@auth.login_required
def get_movFechaBdestBorigTTime():
    
    global invitado
    if request.method == 'GET':
        fecha = request.args.get('fecha')
        bsorigen = request.args.get('bsorigen')
        bsdestino = request.args.get('bsdestino')
        max_dur = request.args.get('maxdur')
        min_dur = request.args.get('mindur')
        
        if not fecha:
            return "Introduce una fecha en la URL", 400
        
        if int(max_dur) - int(min_dur) < 0:
            return "La duracion max debe ser mayor que la duracion min", 400
        
        if invitado == True:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' AND idplug_station = '{bsdestino}' AND travel_time >= '{min_dur}' AND travel_time <= '{max_dur}' LIMIT 10"
        else:
            query = f"SELECT * FROM bicimad WHERE fecha = '{fecha}' AND idunplug_station = '{bsorigen}' AND idplug_station = '{bsdestino}' AND travel_time >= '{min_dur}' AND travel_time <= '{max_dur}'"
        
        df = pd.read_sql_query(query, publicConnection)
        result_json = df.to_json(orient='records', date_format='iso')
        out = jsonify(result_json), 200
        
    return out

In [ ]:
if __name__ == '__main__':
    app.run(port=6878)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6878
Press CTRL+C to quit
127.0.0.1 - - [04/Mar/2024 08:36:20] "GET /api/v1.0/BSOrigen HTTP/1.1" 401 -


True


C:\Users\Daniel\AppData\Local\Temp\ipykernel_26100\2657198569.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, publicConnection)
127.0.0.1 - - [04/Mar/2024 08:36:25] "GET /api/v1.0/BSOrigen HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2024 08:37:21] "GET /api/v1.0/BSOrigen HTTP/1.1" 401 -


False


C:\Users\Daniel\AppData\Local\Temp\ipykernel_26100\2657198569.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, publicConnection)
127.0.0.1 - - [04/Mar/2024 08:37:26] "GET /api/v1.0/BSOrigen HTTP/1.1" 200 -
